In [1]:
import jax
import jax.numpy as jnp
from tqdm import tqdm 
import pandas as pd
import wandb
#import argparse
import scipy
import pickle 
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.backends.backend_pdf as backend_pdf
import subprocess
import time
import glob 
import arviz
import seaborn as sns
from scipy.stats import gaussian_kde

#from bridge_sampling.BFFG import backward_filter, forward_guide, forward_guide_edge, get_logpsi
#from bridge_sampling.setup_SDEs import Stratonovich_to_Ito, dtsdWsT, dWs
#from bridge_sampling.noise_kernel import Q12
from bridge_sampling.helper_functions import *

from mcmc import *
import subprocess
import time

In [2]:
def run_mcmc_for_all_datasets(experiment_path, 
                              num_chains=3, 
                              num_samples=6000, 
                              dt=0.05, 
                              lambd=0.8, 
                              obs_var=0.001, 
                              rb=2, 
                              prior_sigma_min=0.0, 
                              prior_sigma_max=1.0, 
                              prior_alpha_min=0.0, 
                              prior_alpha_max=0.03, 
                              proposal_sigma_tau=0.2, 
                              proposal_alpha_tau=0.015, 
                              use_wandb=True):
    """Run MCMC for all datasets in the experiment path"""

    
    # Get all dataset folders
    dataset_folders = glob.glob(f"{experiment_path}/seed=*")
    
    print(f"Found {len(dataset_folders)} datasets in {experiment_path}")
    
    # Loop through each dataset folder
    for dataset_folder in dataset_folders:
        # Extract the folder name
        folder_name = os.path.basename(dataset_folder)
        print(f"\nProcessing dataset: {folder_name}")
        
        # Check if the required file exists
        data_file = f"{dataset_folder}/procrustes_aligned.csv"
        if not os.path.exists(data_file):
            print(f"  Skipping: {data_file} not found")
            continue
        
        # Generate a random seed for this batch of chains
        seed_start = np.random.randint(0, 1000_000_000)
        
        # Set up output path
        output_path = f"{dataset_folder}/mcmc_seed={seed_start}_N={num_samples}"
        
        print(f"  Starting {num_chains} MCMC chains with seed {seed_start}")
        
        # Start MCMC chains in screen sessions
        screen_sessions = run_mcmc_in_screens(
            num_chains=num_chains,
            script_path="run_mcmc.py",
            seed_param="--seed_mcmc",
            seed_start=seed_start,
            screen_prefix=f"mcmc_{folder_name}",  # Use unique screen names
            script_args={
                "--outputpath": output_path,
                "--phylopath": "../data/chazot_subtree_rounded.nw",
                "--datapath": data_file,
                "--dt": dt,
                "--lambd": lambd,
                "--obs_var": obs_var,
                "--rb": rb,
                "--N": num_samples,
                "--prior_sigma_min": prior_sigma_min,
                "--prior_sigma_max": prior_sigma_max,
                "--prior_alpha_min": prior_alpha_min,
                "--prior_alpha_max": prior_alpha_max,
                "--proposal_sigma_tau": proposal_sigma_tau,
                "--proposal_alpha_tau": proposal_alpha_tau,
                "--use_wandb": True
            }
        )
        
        print(f"  Started chains for {folder_name}. Screen sessions: {', '.join(screen_sessions)}")
        
        # Optional: Add a delay between datasets to avoid overloading the system
        time.sleep(5)
    
    print(f"\nMCMC chains started for all datasets in {experiment_path}")



In [7]:
# MCMC settings
num_chains = 3
num_samples = 6000
dt = 0.05
lambd = 0.7
obs_var = 0.001
rb = 2
prior_sigma_min = 0.0
prior_sigma_max = 2.5
prior_alpha_min = 0.0
prior_alpha_max = 0.03
proposal_sigma_tau = 0.2
proposal_alpha_tau = 0.015
seed_start = np.random.randint(0,1000_000_000)

In [8]:
# Run the function
experiment_path = "exp_1_sigma=0.9_alpha=0.025_dt=0.05"
run_mcmc_for_all_datasets(experiment_path=experiment_path,
                          num_chains=num_chains,
                          num_samples=num_samples,
                          dt=dt,
                          lambd=lambd,
                          obs_var=obs_var,
                          rb=rb,
                          prior_sigma_min=prior_sigma_min,
                          prior_sigma_max=prior_sigma_max,
                          prior_alpha_min=prior_alpha_min,
                          prior_alpha_max=prior_alpha_max,
                          proposal_sigma_tau=proposal_sigma_tau,
                          proposal_alpha_tau=proposal_alpha_tau)    

Found 1 datasets in exp_1_sigma=0.9_alpha=0.025_dt=0.05

Processing dataset: seed=999303364
  Starting 3 MCMC chains with seed 125227906
Starting chain 1 with seed 125227906 in screen 'mcmc_seed=999303364_1'
Starting chain 2 with seed 125227907 in screen 'mcmc_seed=999303364_2'
Starting chain 3 with seed 125227908 in screen 'mcmc_seed=999303364_3'

3 MCMC chains started in separate screen sessions.
To attach to a screen session: screen -r <screen_name>
To detach from a screen session: Ctrl+A, then D
Screen sessions: mcmc_seed=999303364_1, mcmc_seed=999303364_2, mcmc_seed=999303364_3
  Started chains for seed=999303364. Screen sessions: mcmc_seed=999303364_1, mcmc_seed=999303364_2, mcmc_seed=999303364_3

MCMC chains started for all datasets in exp_1_sigma=0.9_alpha=0.025_dt=0.05


# Visualize results 

In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import glob
import arviz as az
import os
import seaborn

def load_mcmc_results(filepath_pattern):
    """
    Load MCMC results from pickle files matching the given pattern.
    
    Args:
        filepath_pattern: Pattern to match pickle files (e.g., "results/chain_*.pkl")
        
    Returns:
        List of loaded results
    """
    results = []
    for filepath in sorted(glob.glob(filepath_pattern)):
        print(f"Loading {filepath}")
        with open(filepath, 'rb') as f:
            results.append(pickle.load(f))
    return results


In [ ]:
def compute_diagnostics(chain_results, burnin_percent):
    trees = np.array([chain_results[i]['trees'] for i in range(len(chain_results))])
    trees = trees[:, int(burnin_percent*trees.shape[1]):, :, :]  # discard burnin
    rhats = []
    esss = []
    for idx in range(trees.shape[2]):  # calculate for all nodes 
        innernodes = trees[:,:,idx, :]
        keys = list(range(innernodes.shape[2]))
        MCMCres = arviz.convert_to_dataset({k:innernodes[:,:,i] for i,k in enumerate(keys)})
        rhats.append(arviz.rhat(MCMCres).to_array().to_numpy())
        esss.append(arviz.ess(MCMCres).to_array().to_numpy())
    return {'Rhat': np.array(rhats), 'ESS': np.array(esss)}


def plot_traces(results, burnin_percent, node_idx, save_path, diagnostics=None, true_values=None): 
    colors = sns.color_palette('pastel', len(results))
    pdf = backend_pdf.PdfPages(save_path + f'/trace_burnin_percent={burnin_percent}.pdf')
    burnin_end = int(results[0]['trees'].shape[0] * burnin_percent)
    plt.figure(1)

    for idx in node_idx: 
        fig, axes = plt.subplots(nrows=7, ncols=6, figsize=(25,15), sharex=True)
        fig.subplots_adjust(top=0.9)  # Adjust this value between 0 and 1
        if true_values is not None: 
            true_innernode = true_values[idx,:]
        for j in range(len(results)): # loop over chains
            innernode = results[j]['trees'][:,idx, :]
            curcol = colors[j]
            for i, ax in zip(range(innernode.shape[1]), axes.flat): # loop over dimensions
                ax.plot(innernode[burnin_end::,i], color = curcol, alpha=0.5)
                if diagnostics:
                    cur_ess = round(diagnostics['ESS'][idx][i], 2)
                    cur_rhat = round(diagnostics['Rhat'][idx][i], 2)
                if true_values is not None:
                    ax.hlines(y=true_innernode[i], xmin=0, xmax=innernode.shape[0]-burnin_end, color='skyblue')
                ax.set_title(f'{i}, Rhat={cur_rhat}, ESS: {cur_ess}')
        fig.suptitle(f'Node {idx}', size=25)
        pdf.savefig()
        plt.clf()
    pdf.close();
    

In [ ]:
# define function for plotting parameter traces
def plot_parameter_traces(chain_results, param_names, burnin_percent):
    """
    Plot the traces of MCMC parameters.
    
    Args:
        chain_results: List of MCMC result dictionaries
        param_names: List of parameter names to plot
        burnin_percent: Percentage of samples to discard as burn-in
    """
    burnin_index = int(burnin_percent * len(chain_results[0][param_names[0]]))
    num_params = len(param_names)
    plt.figure(figsize=(15, 5 * num_params))
    
    # compute convergence diagnostics
    sigmas = [chain_results[i]['sigma'] for i in range(len(chain_results)) if chain_results[i] is not None]
    alphas = [chain_results[i]['alpha'] for i in range(len(chain_results)) if chain_results[i] is not None]
    MCMC_result = dict(zip(["alpha", "sigma"], [alphas, sigmas])) 
    parsres = arviz.convert_to_dataset(MCMC_result)
    rhat = arviz.rhat(parsres)
    ess = arviz.ess(parsres)
    
    for i, param in enumerate(param_names):
        plt.subplot(num_params, 1, i + 1)
        for j in range(len(chain_results)):
            if chain_results[j] is not None:
                plt.plot(chain_results[j][param][burnin_index:], label=f'Chain {j}')
        plt.xlabel('Iteration')
        plt.ylabel(param)
        plt.title(f'MCMC Trace for {param} (R-hat: {rhat[param]:.2f}, ESS: {ess[param]:.1f})')
        plt.legend()
    
    plt.tight_layout()
    plt.show()
    
def plot_log_posterior(result, burnin_percent): 
    """
    Plot the posterior distribution of the MCMC results.
    
    Args:
        result: MCMC result dictionary
        burnin_percent: Percentage of samples to discard as burn-in
    """
    # Compute burn-in index
    burnin_index = int(burnin_percent * len(result[0]['log_posterior']))
    
    # Plot log posterior
    plt.figure(figsize=(10, 5))
    [plt.plot(result[i]['log_posterior'][burnin_index:], label=f'Chain {i}')
     for i in range(len(result))]
    plt.xlabel('Iteration')
    plt.ylabel('Log Posterior')
    plt.title('MCMC Log Posterior Trace')
    plt.legend()
    plt.show()
    
def plot_samples_from_posterior(chain_results, burnin_percent, node_idx, sample_every, savepath, true_values=None):
    pdf = backend_pdf.PdfPages(savepath + f'/samples-posterior-sample_n={sample_every}_burnin_percent={burnin_percent}.pdf')
    
    burnin_end = int(chain_results[0]['trees'].shape[0] * burnin_percent)
    
    for idx in node_idx: # loop over innernodes
        fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(5,5))
        
        # Get all samples from all chains
        all_samples = []
        for j in range(len(chain_results)):
            # Make sure we have data after burnin
            post_burnin_data = chain_results[j]['trees'][burnin_end:, idx, :]
            #print(post_burnin_data)
            
            if len(post_burnin_data) > 0:
                thinned_data = post_burnin_data[0::sample_every, :]
                all_samples.append(thinned_data)
        
        # Combine all chains
        if all_samples:
            innernodes = np.vstack(all_samples)
            
            # Debug info
            print(f"Node {idx}: Found {len(innernodes)} samples, shape={innernodes.shape}")
            
            # Close the shape by appending first landmarks
            inode = np.append(innernodes, innernodes[:, 0:2], axis=1)
            
            # Plot each shape sample
            for i in range(inode.shape[0]):
                x_coords = inode[i, ::2]  # Every other element, starting at 0 (x coordinates)
                y_coords = inode[i, 1::2]  # Every other element, starting at 1 (y coordinates)
                axes.plot(x_coords, y_coords, '--.', color='steelblue', alpha=0.3)
            
            # Add true values if provided
            if true_values is not None:
                true_innernode = true_values[idx, :]
                tinode = np.concatenate((true_innernode, true_innernode[0:2]))  
                axes.plot(tinode[::2], tinode[1::2], '--.', color='black', linewidth=2, label='true shape')
            
            # Add title and format
            fig.suptitle(f'Node {idx}', size=25)
            fig.tight_layout()
            axes.set_aspect('equal')  # Equal aspect ratio
            axes.grid(True, alpha=0.3)
            
            # Save the figure
            pdf.savefig(fig)
            
        plt.close(fig)  # Close the figure to free memory
    
    pdf.close()  # Close the PDF
    print(f"Saved plots to {savepath}/samples-posterior-sample_n={sample_every}_burnin_percent={burnin_percent}.pdf")



def gaussian_kde_contour(x, y, levels=[0.05]):
    
    # Create kernel density estimate
    xy = np.vstack([x, y])
    kde = gaussian_kde(xy)

    # Create a grid to evaluate the KDE
    x_min, x_max = x.min(), x.max()
    y_min, y_max = y.min(), y.max()
    x_range = x_max - x_min
    y_range = y_max - y_min
    x_min -= 0.1 * x_range
    x_max += 0.1 * x_range
    y_min -= 0.1 * y_range
    y_max += 0.1 * y_range
    x_grid, y_grid = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]
    positions = np.vstack([x_grid.ravel(), y_grid.ravel()])

    # Evaluate the KDE at the grid positions
    z = kde(positions).reshape(x_grid.shape)

    # Sort density values and find thresholds for specified levels
    z_flat = z.flatten()
    z_sorted = np.sort(z_flat)
    cumsum = np.cumsum(z_sorted)
    cumsum /= cumsum[-1]  # Normalize


    threshold_idx = np.searchsorted(cumsum, levels)
    threshold = z_sorted[threshold_idx]

    return x_grid, y_grid, z, threshold

In [ ]:
burnin_percent = 0.3
results_path = "exp_1_sigma=0.7_alpha=0.025_dt=0.05/seed=121197884/mcmc_seed=872193302_N=5/results_*.pkl"  # Adjust pattern as needed
save_path = "exp_1_sigma=0.7_alpha=0.025_dt=0.05/seed=121197884/mcmc_seed=872193302_N=5"
sim_path = "exp_1_sigma=0.7_alpha=0.025_dt=0.05/seed=121197884"
chain_results = load_mcmc_results(results_path)
param_names = ["sigma", "alpha"]  # Replace with your actual parameter names
len(chain_results)

In [ ]:
plot_log_posterior(chain_results, burnin_percent)

In [ ]:
[np.mean(result['acceptsigma']) for result in chain_results if result is not None]

In [ ]:
[np.mean(result['acceptalpha']) for result in chain_results if result is not None]

In [ ]:
[np.mean(result['acceptpath']) for result in chain_results if result is not None]

In [ ]:
plot_parameter_traces(chain_results, param_names, burnin_percent)

In [ ]:
diagnostics = compute_diagnostics(chain_results, burnin_percent)

In [ ]:
plt.plot(chain_results[0]['trees'][:,0,0])

In [ ]:
plot_traces(chain_results, burnin_percent, node_idx=[0, 1, 2, 6], save_path=save_path, diagnostics=diagnostics)

In [ ]:
plot_samples_from_posterior(chain_results, burnin_percent=0.5, node_idx=[0, 1, 2, 6], sample_every=50, savepath=save_path, true_values=None)

## Look at posterior covariation 

In [ ]:
# visualize landmarks with labels 
flat_true_tree = np.genfromtxt(sim_path + "/flat_true_tree.csv", delimiter=',')   
plt.plot(flat_true_tree[0][0::2], flat_true_tree[0][1::2], '.', color='black', linewidth=2, label='true shape')
n = list(range(len(flat_true_tree[0][::2])))
for i, txt in enumerate(n):
    plt.annotate(txt, (flat_true_tree[0][::2][i], flat_true_tree[0][1::2][i]))

In [ ]:
innernode_python = [0, 1, 2, 6]
innernode_R = [6,7,8,9]
node = 2
dim = [0, 2, 22]

# nchain x mcmc iter x nnodes x ndim
burnin_end = int(chain_results[0]['trees'].shape[0] * burnin_percent)
all_posterior_samples = np.array([chain_results[i]['trees'][burnin_end:7000,:,:] for i in range(len(chain_results)) if chain_results[i] is not None])
l0x = all_posterior_samples[:,:,node,dim[0]].flatten()
l1x = all_posterior_samples[:,:,node,dim[1]].flatten()
l12x = all_posterior_samples[:,:,node,dim[2]].flatten()

# load confidence intervals from folder 
conf_dim0 = pd.read_csv(sim_path + f"/fastAnc/95%_conf_trait{dim[0]+1}.csv", header=0, index_col=0)
conf_dim1 = pd.read_csv(sim_path + f"/fastAnc/95%_conf_trait{dim[1]+1}.csv", header=0, index_col=0)
conf_dim2 = pd.read_csv(sim_path + f"/fastAnc/95%_conf_trait{dim[2]+1}.csv", header=0, index_col=0)

In [ ]:
import matplotlib.patches as patches
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 12), sharex=True, sharey=False)
plt.subplots_adjust(wspace=0.3, hspace=0.3)
axes = axes.flatten()

axes[0].set_xlabel('Landmark 0 (x-coordinate)')
axes[0].set_ylabel('Landmark 1 (x-coordinate)')
axes[0].set_title('Brownian motion model', size=15)
x_min, x_max = conf_dim0.iloc[0, 0], conf_dim0.iloc[0, 1]; y_min, y_max = conf_dim1.iloc[0, 0], conf_dim1.iloc[0, 1]
padding = 0.2
x_range = x_max - x_min
y_range = y_max - y_min
axes[0].set_xlim(x_min - padding * x_range, x_max + padding * x_range)
axes[0].set_ylim(y_min - padding * y_range, y_max + padding * y_range)
rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, 
                         linewidth=1, 
                         edgecolor='black', 
                         facecolor='steelblue', 
                         alpha=0.5)
axes[0].add_patch(rect)
#axes[0].text(x_max, y_min, "Region of interest", 
#             fontsize=9, 
#             color='black', 
#             va='bottom')


axes[1].set_xlabel('Landmark 0 (x-coordinate)')
axes[1].set_ylabel('Landmark 1 (x-coordinate)')
axes[1].set_title('Shape aware model', size=15)
axes[1].plot(l0x, l1x, '.', color='steelblue', alpha=0.3)
x_grid, y_grid, z, threshold = gaussian_kde_contour(l0x, l1x, levels=[0.05])
contour = axes[1].contour(x_grid, y_grid, z, levels=[threshold], 
                         colors='red', linewidths=2)
#axes[1].clabel(contour, inline=True, fontsize=10, fmt='%.2f')

axes[2].set_xlabel('Landmark 0 (x-coordinate)')
axes[2].set_ylabel('Landmark 12 (x-coordinate)')
x_min, x_max = conf_dim0.iloc[0, 0], conf_dim0.iloc[0, 1]; y_min, y_max = conf_dim2.iloc[0, 0], conf_dim2.iloc[0, 1]
axes[2].set_xlim(x_min - padding * x_range, x_max + padding * x_range)
axes[2].set_ylim(y_min - padding * y_range, y_max + padding * y_range)
rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, 
                         linewidth=1, 
                         edgecolor='black', 
                         facecolor='steelblue', 
                         alpha=0.5)
axes[2].add_patch(rect)

axes[3].set_xlabel('Landmark 0 (x-coordinate)')
axes[3].set_ylabel('Landmark 12 (x-coordinate)')
axes[3].plot(l0x, l12x, '.', color='steelblue', alpha=0.3)
x_grid, y_grid, z, threshold = gaussian_kde_contour(l0x, l12x, levels=[0.05])
contour = axes[3].contour(x_grid, y_grid, z, levels=[threshold], 
                         colors='red', linewidths=2)


In [ ]:
y_min

In [ ]:
y_max